# ICEL9 Data Analysis Workshop
Python version - Christoffer Roepstorff, collaboration with J.I.M Parmentier

### Skeleton of the assignment - part 1 (Looking at 1 file)
1. Declare the datapath and different needed information (e.g., horses identifiers, markers of interest) and load the file
2. Extract information (labels (marker names), markers' trajectories, sampling rate, measurement duration), create a time vector, extract the trajectories of interest
3. Visualise the data, see that some data have missing samples (Motion Capture)
4. Deal with missing samples, if missing samples are less than 3 
5. Extract the segment of interest (no missing data)
6. Explore the effect of mean removal, fitlering 
7. Stride split, based on LH hoof or LH fetlock (why LH? discuss it during the workshop)
8. Extract the parameters of interest
9. Save the parameters in a table (e.g., csv file)

## I. Load data and declare variables of interest
Before any data is loaded we need to import the packages we are going to use in this notebook.

In [ ]:
import pandas as pd  # Pandas helps us handle tabular data and load the .csv files
import numpy as np  # Matrix manipulation and math library with matlab similar syntax
import ipywidgets as widgets  # Widgets for this notebook
import plotly.graph_objects as go  # Used for plotting
from plotly.subplots import make_subplots  # Used for plotting
import plotly
import scipy as sc
from pathlib import Path  # Handles directory and file paths for different operating systems

To load the data, you need to first define a datapath. You can copy-paste below where your data is located (the relative path to the repo data folder should work out of the box). We will use our first imported library to load the csv file of **horseA** at time point **TO**.:

In [ ]:
# Specify the data path
datapath = Path("../data")

# Specify horse name and the time point 
horse_name = "horseA"
time_point = "T0"

# Create file name variable use the Path to create filepaths that work on any typ of operating system (Windows, Mac or Linux)
file_name = datapath / Path(f"{time_point}_{horse_name}_trot1.csv")

# Create a pandas dataframe, a dataframe holds tabular data
df = pd.read_csv(file_name)

# Print the data frame to show a cropped version of the table
display(df)


As you can see from the displayed data frame abova each column holds a timeseries of data for a motion capture marker along one axis. For example, 

column 1: The `Poll` markers `x` position, i.e., the movement along the x-axis (cranio-caudal)

column 2 : The `Poll` markers `y` position, i.e., the movement along the y-axis (medio-lateral)

column 3 : The `Poll` markers `z` position, i.e., the movement along the z-axis (vertical)

etc

We will work with three upper-body markers: Head (`Poll`), Withers (`T8`) and the Pelvis (`TubSac`) and two stride split markers hoof (`Hoof_LH`) and fetlock (`fetl_HL`). Next we will define the marker and axis labels so that we can extract timeseries data from our [dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). 


In [ ]:
markers_upper_body = ['Poll','T8','TubSac']   # Declare upper body markers
markers_stride_split = ['Hoof_LH','Fetl_LH']  # Declare stride split markers
axis = ["x", "y", "z"]  # Declare the cartesian coordinate identifiers
axis_description = [""]

## II. Extract the trajectories
Using our newly declared variables we can create strings by concatenating marker and axis strings and use these select the timeseries we want from our dataframe. The last column in th data frame hold the framerate we will use this to create a time vector.

<div class="alert alert-block alert-info">
<b>Note:</b><br>In the code below we will also use some code to create widgets for selecting different marker and axis combinations. This way you don't have to change any code but can just use the created dropdown instead.
</div>

In [ ]:
# Save the framerate, the '0' here means we only extract the values from the first frame of the data
fs = df["FrameRate"][0] 

# Now we can create a time vector, the same length as our timeseries data (the number of rows in the dataframe)
time = np.linspace(0, (len(df)-1)/fs, len(df)) 

# Create widgets
upbm_dropdown = widgets.Dropdown(options=markers_upper_body, value=markers_upper_body[0], description='Upper body marker:', style={'description_width': 'initial'})
a_dropdown = widgets.Dropdown(options=axis, value=axis[2], description='Axis:', style={'description_width': 'initial'})


Select marker and axis:

In [ ]:
display(upbm_dropdown)
display(a_dropdown)

## III. Plot the trajectories of interest
Visualizing data is always important. In this segment we will showcase a few examples of how timeseries data can be plotted.

### Single trajectory and axis plot (uses the dropdown selection above)

In [ ]:
# Get trajectory selection from widgets
selected_axis = a_dropdown.value
selected_marker = upbm_dropdown.value
selected_trajectory = df[f"{selected_marker}_{selected_axis}"]

# Create a figure and a trace
fig = go.Figure().add_trace(go.Scatter(x=time, y=selected_trajectory, name=f"{selected_marker} {selected_axis}"))
fig.update_layout(xaxis_title="Time (s)", yaxis_title="Amplitude (mm)", title=f"{selected_axis}-axis displacement of the {selected_marker} marker")
fig.show()


### Plotting multiple markers (uses axis selection dropdown above)

In [ ]:
# Get trajectory selection from widgets
selected_axis = a_dropdown.value

# Create a figure and a trace
fig = go.Figure()
for marker in markers_upper_body:
    selected_trajectory = df[f"{marker}_{selected_axis}"]
    fig.add_trace(go.Scatter(x=time, y=selected_trajectory, name=f"{marker} {selected_axis}"))

fig.update_layout(xaxis_title="Time (s)", yaxis_title="Amplitude (mm)", title=f"{selected_axis}-axis displacement of {markers_upper_body} markers")
fig.show()

### All markers and axis using subplots

In [ ]:
# Create subplot figure
fig = make_subplots(rows=len(markers_upper_body), cols=1, subplot_titles=markers_upper_body)

# Loop over both markers and axis
for marker_cnt, marker in enumerate(markers_upper_body):
    for ax in axis:
        selected_trajectory = df[f"{marker}_{ax}"]
        fig.add_trace(go.Scatter(x=time, y=selected_trajectory, name=f"{marker} {ax}"), row=marker_cnt+1, col=1)

        fig.update_yaxes(title="Amplitude (mm)", row=marker_cnt+1, col=1)
fig.update_xaxes(title="Time(s)", row=len(markers_upper_body), col=1)
fig.show()

## IV. Deal with missing samples
When you look at the `Poll` marker plot above you notice that there are samples that have missing values (look between ~4 and ~6 seconds). One way to deal with missing samples is interpolation, in this section we explore a few different type of interpolation methods and look at their effects on the data.

In [ ]:
# Import local functions
from missing_samples import fill_missing, InterpolationEnum

# Create widget for interpolation options
fm_dropdown = widgets.Dropdown(options=[opt.name for opt in InterpolationEnum], value=InterpolationEnum.nearest.name, description='Interpolation method:', style={'description_width': 'initial'})
display(fm_dropdown)

# Slider for selecting max gap length
ml_slider = widgets.IntSlider(value=30, min=0, max=fs, step=1, description='Max gap-length (#frames):', orientation='horizontal', readout=True, readout_format='d')
display(ml_slider)

# Create widget for degree (used by the spline method)
d_dropdown = widgets.Dropdown(options=[1, 2, 3, 4, 5], value=2, description="Degree (used by 'spline' method)", style={'description_width': 'initial'})
display(d_dropdown)

In [ ]:
# Get selected interpolation
selected_interpolation = InterpolationEnum[fm_dropdown.value]

# # Get poll vertical displacement
z_raw = df["Poll_z"]
# 
z_filled, z_fill_only = selected_interpolation.fill_missing(data=z_raw.to_numpy(), k=d_dropdown.value, max_gap_length=ml_slider.value)
fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=z_filled, name="filled"))
fig.add_trace(go.Scatter(x=time, y=z_fill_only, name="fill"))

fig.update_layout(yaxis_title="Amplitude (mm)", xaxis_title="Time (s)")
fig.show()

## V. Extract segement without missing data
It is possible to gapfill any length of missing data but that migh not always be the best alternative. It is sometimes better to only select a range of data that is entirely finite. Select a range of the `Poll` data that has no missing frames.

In [ ]:
# Create range slider
ra_slider = widgets.FloatRangeSlider(value=[5.3, 11.5], min=0, max=np.max(time), step=0.01, description='Select finite range:', orientation='horizontal',readout=True,readout_format='.2f', style={'description_width': 'initial'})
display(ra_slider)

Below is a visualization of the selected range for `z-axis` data for the upper body

In [ ]:
# Calculate start and end frames based on range selection
start_index = np.argmin(np.abs(time-ra_slider.value[0]))
end_index = np.argmin(np.abs(time-ra_slider.value[1]))

# Plot selected segment upper body
fig = make_subplots(rows=len(markers_upper_body), cols=1, subplot_titles=markers_upper_body)

# Loop over markers
for marker_cnt, marker in enumerate(markers_upper_body):
    selected_trajectory = df[f"{marker}_z"]
    fig.add_trace(go.Scatter(x=time[start_index:end_index], y=selected_trajectory[start_index:end_index], name=f"{marker} z"), row=marker_cnt+1, col=1)

    fig.update_yaxes(title="Amplitude (mm)", row=marker_cnt+1, col=1)
    
fig.update_xaxes(title="Time(s)", row=len(markers_upper_body), col=1)
fig.show()

Below are the plots for the `z-axis` hoof and fetlock data

In [ ]:
fig = make_subplots(rows=len(markers_upper_body), cols=1, subplot_titles=markers_stride_split)

# Loop over markers
for marker_cnt, marker in enumerate(markers_stride_split):
    selected_trajectory = df[f"{marker}_z"]
    fig.add_trace(go.Scatter(x=time[start_index:end_index], y=selected_trajectory[start_index:end_index], name=f"{marker} z"), row=marker_cnt+1, col=1)

    fig.update_yaxes(title="Amplitude (mm)", row=marker_cnt+1, col=1)
    
fig.update_xaxes(title="Time(s)", row=len(markers_stride_split), col=1)
fig.show()

## VI. Prepare your data with mean removal, filtering, etc.
Now that you have extracted a segment of interest, we can prepare it for data analysis. Usually, we remove:
 - movement artifacts (e.g., large head movements)
 - Noise (e.g., noise related to the data collection process)
 - Others
 
Today, we are going to look at the effect of different filters on the data. First we begin by removbing the mean and plotting the *amplitude spectrum* of the `Poll z` data, that does not have any missing values.

The amplitude spectrum can be plotted by using the Fast Fourier Transform (FFT)
<div class="alert alert-block alert-info">
<b>Note:</b><br>The FFT will not work on data has has missing values in it (nan for exmaple). Neither will it work properly unless we remove the mean or detrend the data. Very large movement (like can be seen in the poll movement of horseA T0) can also disturb.
</div>

In [ ]:
# Get the poll z segment from the dataframe and index using the defined start and end frames 
z_poll = df[f"Poll_z"].to_numpy()
z_poll_segment = z_poll[start_index:end_index]
time_segment = time[start_index:end_index]

# Get signal length
L = len(z_poll_segment);

# FFT
z_poll_segment_detrend = sc.signal.detrend(z_poll_segment)
Y = np.fft.fft(z_poll_segment_detrend);

# Extract the amplitude spectrum ba taking the abnsolute values of the complex valued transform
amp = 2*np.abs(Y)/L;

# Create a vector with frequencies
f = fs*np.linspace(0,1,L);

# Plot the detrended signal and the frequency spectrum
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=time_segment, y=z_poll_segment_detrend, name="Detrended signal"), row=1, col=1)
fig.update_xaxes(title="Time (s)", row=1, col=1)
fig.update_yaxes(title="Amplitude (mm)", row=1, col=1)

# Plot the amplitude spectrum
fig.add_trace(go.Scatter(x=f, y=amp, name="Amplitude spectrum"), row=2, col=1)
fig.update_xaxes(title="Frequency (Hz)", row=2, col=1)
fig.update_yaxes(title="Amplitude (mm)", row=2, col=1)

# Limit the plot range to only show the interesting frequencies
fig.update_xaxes(range=[0, 10], row=2, col=1)
fig.show()


Do you see a clear and distinctive peak around 3.5Hz? Now lets try and apply a filter to this segment, select a range of freqeuncies to include

In [ ]:
# Create range slider
raf_slider = widgets.FloatRangeSlider(value=[0.1, 9], min=0.01, max=10, step=0.01, description='Frequency range:', orientation='horizontal',readout=True,readout_format='.2f', style={'description_width': 'initial'})
display(raf_slider)

# Create order Int slider
oi_slider = widgets.IntSlider(value=4, min=1, max=10, step=1, description='Filter order:', orientation='horizontal',readout=True,readout_format='.2f', style={'description_width': 'initial'})
display(oi_slider)

In [ ]:
# Low and high cut-off frequencies from slider selection
order = oi_slider.value
low, high = raf_slider.value 

# Create a function that can be called a little easier
def sos_bandpass_filter(data: np.ndarray, frame_rate: float, low_cut: float, high_cut: float, 
                        order: int) -> np.ndarray:
    
    # Make the cutoffs reltive to nyqist frequency
    low_n = low_cut/frame_rate*2
    high_n = high_cut/frame_rate*2

    # Create filter
    sos = sc.signal.butter(order, [low_n, high_n], btype='band', output='sos')
    
    # Filter and return result
    return sc.signal.sosfiltfilt(sos, data)

# Filter the signal
z_poll_filtered = sos_bandpass_filter(data=z_poll_segment, frame_rate=fs, low_cut=low, high_cut=high, order=order)

# Plot the filtered signal and 
fig = go.Figure()
fig.add_trace(go.Scatter(x=time_segment, y=z_poll_filtered, name="Filtered signal"))
fig.update_xaxes(title="Time (s)")
fig.update_yaxes(title="Amplitude (mm)")
fig.update_layout(title=f"Filtered signal band-pass ({low:0.1f} - {high:0.1f} Hz)")
fig.show()


## VII. Stride Splitting
Since trot is a cyclical movement, we can use limb data to stride-split. Conventionally, the different research groups have agreed to base the stride splitting on the left hind limb.
If sensors or markers are available on the hoof, you can use these. Otherwise, you can use the metatarsus marker, or even the pelvis markers.

In this example we will use either hoof or fetlock z-position (vertical movement),

In [ ]:
# Create widget for stride split selection
sp_dropdown = widgets.Dropdown(options=markers_stride_split, value=markers_stride_split[0], description='Select marker for stride split:', style={'description_width': 'initial'})
display(sp_dropdown)

In [ ]:
# Get the stridesplit timeseries, get marker from dataframe and directy slice to inlcude only the selected segment
sp_timeseries = df[f"{sp_dropdown.value}_z"].to_numpy()[start_index:end_index]

# Calculate the difference ie.e, the acceleration
sp_acceleration = np.gradient(np.gradient(sp_timeseries/1000, 1/fs), 1/fs)

# Plot the result
fig = go.Figure()
fig.add_trace(go.Scatter(x=time_segment, y=sp_timeseries))
fig.update_xaxes(title="Time (s)")
fig.update_yaxes(title="Amplitude (mm)")
fig.update_layout(title=f"{sp_dropdown.value} vertical displacement, frame {start_index} to {end_index}")
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=time_segment, y=sp_acceleration))
fig.update_xaxes(title="Time (s)")
fig.update_yaxes(title="Acceleration (m/s²)")
fig.update_layout(title=f"{sp_dropdown.value} vertical speed, frame {start_index} to {end_index}")
fig.show()

Looking at the curves in the figures above we notice that there are cyclic occurences of minmas and maximas in the postion and spikes in the differentiated curve as well. 

One common way to extract features, in this case find indexes in that that split strides, is to look for peaks in the data. The scipy package has a nice function that takes some interesting arguments, we will look at two of them.

1. **distance** set a minimum number of frames required between located peaks
2. **prominence** require the peak to 'stand out' relative to its surroundings

It appears as the differentiated signal has more clear spikes that seem to coincide with the timing of the stance phase. We will try to extract the strides from the acceleration signal.

In [ ]:
fp_dist = widgets.FloatText(value=90, description='distance:', style={'description_width': 'initial'})
display(fp_dist)

fp_prom = widgets.FloatText(value=90, description='prominence:', style={'description_width': 'initial'})
display(fp_prom)

<div class="alert alert-block alert-info">
    <b>Note:</b><br>In the matlab version of this workshop a function called <code>ischange</code>. Python can achieve similar functionality by installing a package called ruptures (<code>pip install ruptures</code>). If you install this in the enviroment you can uncomment ruptures part below to have test it out.
</div>

In [ ]:
# Use the findpeaks function with a distance and a prominence argument
peak_indices, other = sc.signal.find_peaks(sp_acceleration, distance=fp_dist.value, prominence=fp_prom.value)

## Change indices extraction using an external package
# import ruptures as rpt
# algo = rpt.Pelt(model="normal").fit(sp_timeseries)
# result = algo.predict(pen=100)
# peak_indices = result[0:-1]
# print(result)
# print(peak_indices)

# Get available colors in plotly
colors = plotly.colors.qualitative.Plotly

# Plot the peaks 
fig = go.Figure()
fig.add_trace(go.Scatter(x=time_segment, y=sp_acceleration, name=f"{sp_dropdown.value} vertical acceleration"))
fig.add_trace(go.Scatter(x=time_segment[peak_indices], y=sp_acceleration[peak_indices], mode="markers", name="peaks"))

fig.update_xaxes(title="Time (s)")
fig.update_yaxes(title="Acceleration (m/s²)")
fig.update_layout(title=f"{sp_dropdown.value} vertical speed, frame {start_index} to {end_index} distance: {fp_dist.value:.0f} prominence: {fp_prom.value}")
fig.show()

# Plot the position
fig = go.Figure()
fig.add_trace(go.Scatter(x=time_segment, y=sp_timeseries, name=f"{sp_dropdown.value} vertical movement", showlegend=True))

for peak_ind in peak_indices:
    fig.add_vline(x=time_segment[peak_ind], line_width=1, line_dash="dash", line_color=colors[1])
    
fig.update_xaxes(title="Time (s)")
fig.update_yaxes(title="Amplitude (mm)")
fig.update_layout(title=f"{sp_dropdown.value} vertical movement, frame {start_index} to {end_index} distance: {fp_dist.value:.0f} prominence: {fp_prom.value}")
fig.show()

# Show the stride split on some filtered head data
fig = go.Figure()
fig.add_trace(go.Scatter(x=time_segment, y=z_poll_filtered, name=f"Poll filtered position", showlegend=True))

for peak_ind in peak_indices:
    fig.add_vline(x=time_segment[peak_ind], line_width=1, line_dash="dash", line_color="red")

fig.update_xaxes(title="Time (s)")
fig.update_yaxes(title="Amplitude (mm)")
fig.update_layout(title=f"Filtered Poll vertical movement")
fig.show()


## VIII Extract the variables
Now that the stride split has been performed we can start to extract variables from the strides, we will focus on 3 metrics here,

- stride duration
- minDiff
- maxDiff

MinDiff is the difference of minimum reached during the left stance minus the minimum reached during the right stance.

MaxDiff is the difference of maximum reached during the left stance minus the maximum reached during the right stance.

For example, if the head reaches a minimum of -20mm during the left stance and -10mm during the right stance, the Head MinDiff is of (-20mm) - (-10mm) = -10mm.

Thus, if a Head MinDiff is negative, the head is going lower during the left stance than during the right stance, indicating a right front limb asymmetry.

Since we cut strides based on LH-on, the first valley/peak correspond to the impact/push-off of LH-RF respectively. 

Keep this in mind when defining which peak to substract to the other!

Below we show the process for the `Poll` marker,

In [ ]:
# Loop over our stride split and apply it to the extracted finite segment of poll vertical position.
# Find the minimas and  maximas per stride and calculate differences. Alos calculate the duration of each stride

# Prepare figure
fig = go.Figure()
fig2 = go.Figure()

# Prepare output
variables = dict(stride_duration=[], min_diff=[], max_diff=[])

# Loop over all peak indices except the last one (-1) as a stride happens between indices
for peak_cnt in range(len(peak_indices) - 1):
    stride_start = peak_indices[peak_cnt]
    stride_end = peak_indices[peak_cnt + 1]
    stride_ts = z_poll_filtered[stride_start:stride_end]
    
    # Calculate stride duration
    stride_duration = (stride_end - stride_start)/fs
    variables["stride_duration"].append(stride_duration)
    
    # Find peaks and troughs/valleys
    stride_peaks, _ = sc.signal.find_peaks(stride_ts)
    stride_troughs, _ = sc.signal.find_peaks(-stride_ts)
    
    # Calculate min diff
    if len(stride_troughs) == 2:
        mindiff = stride_ts[stride_troughs[1]] - stride_ts[stride_troughs[0]]
    else:
        mindiff = np.nan
        
    # Calculate max diff
    if len(stride_peaks) == 2:
        maxdiff = stride_ts[stride_peaks[1]] - stride_ts[stride_peaks[0]]
    else:
        maxdiff = np.nan
    
    variables["min_diff"].append(mindiff)
    variables["max_diff"].append(maxdiff)

    fig.add_trace(go.Scatter(y=stride_ts, name=f"Poll z stride {peak_cnt}"))

fig.update_xaxes(tickvals = [0, 25, 50, 75, 100],
                 ticktext = ['0%', '25% - Left hind stance', '50%', '75% - Right hind stance', '100%'])
fig.update_layout(title="Poll verical displacement strides", yaxis_title="Amplitude (mm)")
fig.show()

fig2.add_trace(go.Box(y=variables["min_diff"], name="Min diff", boxpoints='all'))
fig2.add_trace(go.Box(y=variables["max_diff"], name="Max diff", boxpoints='all'))
fig2.update_layout(title="Min and max diffs for Poll", yaxis_title="Aymmetry (mm)")
fig2.show()

## IX.  Create a table and export to csv
Once the entire chain of processing has been done it is good to save your results. How you want to do this naturally depends on the results structure, but exporting to some form of table format is common. Here we will showcase how the results can be converted to a pandas dataframe and exported to.csv

In [ ]:
# Convert variables to pandas dataframe
results_df = pd.DataFrame.from_dict(variables)

# Insert categorical data about horse and timepoint (defined when loading the original data)
results_df.insert(0, horse_name, ["HorseA"]*len(results_df))
results_df.insert(1, "time_point", ["T0"]*len(results_df))

# Display output and do basic statistics
display(results_df)
display(results_df.describe())

# Save to csv
results_file_name = f"{horse_name}_{time_point}_results.csv"
results_df.to_csv(f"{horse_name}_{time_point}_results.csv", index=False) # Use index=false to not save the row count


# Test to load the file 
loaded_results_df = pd.read_csv(results_file_name)
display(loaded_results_df)